In [ ]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 36kB/s 
     |████████████████████████████████| 512kB 33.0MB/s 
     |████████████████████████████████| 3.8MB 36.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=6c6484fd6a010725107bcbea39399ab3ec3a70577d3b8a9dcc26b2ea87c37ba1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: 

In [ ]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
import tensorflow.train as tft
import time
from pathlib import Path
import matplotlib.pyplot as plt
# Authentication for Managing Data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
dataOpen = np.zeros(1)
while not np.any(dataOpen):
    try:
        with open('/content/drive/My Drive/DataRaw/Detection/Training/DataOpenFault4N_11_1.csv','r') as f:
            dataOpen = np.genfromtxt(f,delimiter = ",")
    except:
        pass
np.shape(dataOpen)

(141649, 6)

In [ ]:
def process(fileName):
    # Load correct File
    with tf.io.gfile.GFile(fileName,'r') as f:
        data = np.genfromtxt(f,delimiter = ",")
    # Seperate Relative Position & Velocity data
    info = data[0,:]
    settleIndex = 300
    faultTime = np.ceil(info[0])
    if faultTime<1:
        faultTime = 1e10
    faultSat = info[1]
    faultThruster = info[2]
    faultType = info[3]

    posvelData = data[1:,:]
    noRows = np.size(posvelData, 0)//24 - settleIndex
    noCols = 6
    dataSat1 = np.zeros([noRows, 4*noCols])
    dataSat2 = np.zeros([noRows, 4*noCols])
    dataSat3 = np.zeros([noRows, 4*noCols])
    dataSat4 = np.zeros([noRows, 4*noCols])
    dataSat5 = np.zeros([noRows, 4*noCols])
    dataSat6 = np.zeros([noRows, 4*noCols])
    
    faultLabel = np.zeros([noRows, 1])
    faultLabel[np.where(np.arange(noRows)>faultTime-settleIndex)] = 1
    settleOffset = settleIndex * 24 
    dataSat1[:,0*noCols:1*noCols] = posvelData[settleOffset + 0::24,0:noCols]
    dataSat1[:,1*noCols:2*noCols] = posvelData[settleOffset + 1::24,0:noCols]
    dataSat1[:,2*noCols:3*noCols] = posvelData[settleOffset + 2::24,0:noCols]
    dataSat1[:,3*noCols:4*noCols] = posvelData[settleOffset + 3::24,0:noCols]

    dataSat2[:,0*noCols:1*noCols] = posvelData[settleOffset + 4::24,0:noCols]
    dataSat2[:,1*noCols:2*noCols] = posvelData[settleOffset + 5::24,0:noCols]
    dataSat2[:,2*noCols:3*noCols] = posvelData[settleOffset + 6::24,0:noCols]
    dataSat2[:,3*noCols:4*noCols] = posvelData[settleOffset + 7::24,0:noCols]

    dataSat3[:,0*noCols:1*noCols] = posvelData[settleOffset + 8::24,0:noCols]
    dataSat3[:,1*noCols:2*noCols] = posvelData[settleOffset + 9::24,0:noCols]
    dataSat3[:,2*noCols:3*noCols] = posvelData[settleOffset + 10::24,0:noCols]
    dataSat3[:,3*noCols:4*noCols] = posvelData[settleOffset + 11::24,0:noCols]

    dataSat4[:,0*noCols:1*noCols] = posvelData[settleOffset + 12::24,0:noCols]
    dataSat4[:,1*noCols:2*noCols] = posvelData[settleOffset + 13::24,0:noCols]
    dataSat4[:,2*noCols:3*noCols] = posvelData[settleOffset + 14::24,0:noCols]
    dataSat4[:,3*noCols:4*noCols] = posvelData[settleOffset + 15::24,0:noCols]

    dataSat5[:,0*noCols:1*noCols] = posvelData[settleOffset + 16::24,0:noCols]
    dataSat5[:,1*noCols:2*noCols] = posvelData[settleOffset + 17::24,0:noCols]
    dataSat5[:,2*noCols:3*noCols] = posvelData[settleOffset + 18::24,0:noCols]
    dataSat5[:,3*noCols:4*noCols] = posvelData[settleOffset + 19::24,0:noCols]

    dataSat6[:,0*noCols:1*noCols] = posvelData[settleOffset + 20::24,0:noCols]
    dataSat6[:,1*noCols:2*noCols] = posvelData[settleOffset + 21::24,0:noCols]
    dataSat6[:,2*noCols:3*noCols] = posvelData[settleOffset + 22::24,0:noCols]
    dataSat6[:,3*noCols:4*noCols] = posvelData[settleOffset + 23::24,0:noCols]

    sats = np.arange(6)
    sats = np.repeat(sats,noRows)
    sats = sats.reshape(-1,1)
    time = np.arange(settleIndex,noRows+settleIndex)

    time = np.tile(time,6).reshape(-1,1)

    data = np.concatenate((dataSat1,dataSat2,dataSat3,dataSat4,dataSat5,dataSat6),0)
    labels = np.concatenate((faultLabel,faultLabel,faultLabel,faultLabel,faultLabel,faultLabel),0)
    data = np.concatenate((data,labels,time,sats),axis = 1)

    return data # ds3.map(lambda a,b,c: (a,b)), ds3.map(lambda a,b,c: c)
 


In [ ]:
def create_tfrecord(filePath,fileName, data):
    # Create tfrecord
    
    header = ['x1','y1','z1','vx1','vy1','vz1',
                'x2','y2','z2','vx2','vy2','vz2',
                'x3','y3','z3','vx3','vy3','vz3',
                'x4','y4','z4','vx4','vy4','vz4',
                'label','time','sat']
    # Create dict
    x1  = tft.Feature(float_list = tft.FloatList(value = data[:,0]))
    y1  = tft.Feature(float_list = tft.FloatList(value = data[:,1]))
    z1  = tft.Feature(float_list = tft.FloatList(value = data[:,2]))
    vx1 = tft.Feature(float_list = tft.FloatList(value = data[:,3]))
    vy1 = tft.Feature(float_list = tft.FloatList(value = data[:,4]))
    vz1 = tft.Feature(float_list = tft.FloatList(value = data[:,5]))

    x2  = tft.Feature(float_list = tft.FloatList(value = data[:,6]))
    y2  = tft.Feature(float_list = tft.FloatList(value = data[:,7]))
    z2  = tft.Feature(float_list = tft.FloatList(value = data[:,8]))
    vx2 = tft.Feature(float_list = tft.FloatList(value = data[:,9]))
    vy2 = tft.Feature(float_list = tft.FloatList(value = data[:,10]))
    vz2 = tft.Feature(float_list = tft.FloatList(value = data[:,11]))

    x3  = tft.Feature(float_list = tft.FloatList(value = data[:,12]))
    y3  = tft.Feature(float_list = tft.FloatList(value = data[:,13]))
    z3  = tft.Feature(float_list = tft.FloatList(value = data[:,14]))
    vx3 = tft.Feature(float_list = tft.FloatList(value = data[:,15]))
    vy3 = tft.Feature(float_list = tft.FloatList(value = data[:,16]))
    vz3 = tft.Feature(float_list = tft.FloatList(value = data[:,17]))

    x4  = tft.Feature(float_list = tft.FloatList(value = data[:,18]))
    y4  = tft.Feature(float_list = tft.FloatList(value = data[:,19]))
    z4  = tft.Feature(float_list = tft.FloatList(value = data[:,20]))
    vx4 = tft.Feature(float_list = tft.FloatList(value = data[:,21]))
    vy4 = tft.Feature(float_list = tft.FloatList(value = data[:,22]))
    vz4 = tft.Feature(float_list = tft.FloatList(value = data[:,23]))

    label = tft.Feature(int64_list = tft.Int64List(value = data[:,24].astype(int)))
    time  = tft.Feature(int64_list = tft.Int64List(value = data[:,25].astype(int)))
    sats  = tft.Feature(int64_list = tft.Int64List(value = data[:,26].astype(int)))



    feature_dict = {'x1':x1,'y1':y1,'z1':z1,'vx1':vx1,'vy1':vy1,'vz1':vz1,
                    'x2':x2,'y2':y2,'z2':z2,'vx2':vx2,'vy2':vy2,'vz2':vz2,
                    'x3':x3,'y3':y3,'z3':z3,'vx3':vx3,'vy3':vy3,'vz3':vz3,
                    'x4':x4,'y4':y4,'z4':z4,'vx4':vx4,'vy4':vy4,'vz4':vz4,
                    'label':label,'time':time,'sats':sats}
    features = tft.Features(feature = feature_dict)
    DataExample = tft.Example(features = features)

    with tf.python_io.TFRecordWriter(filePath+fileName) as writer:
        writer.write(DataExample.SerializeToString())
    return


In [ ]:
# Testing
path = '/content/drive/My Drive/DataRaw/Detection/Testing/'
nfPattern = path + 'DataNoFault4N_*.csv'
openPattern = path + 'DataOpenFault4N_*.csv'
closedPattern= path + 'DataClosedFault4N_*.csv'
globPatterns = [nfPattern,openPattern,closedPattern]
# Seed is 6 DO NOT CHANGE
seed = 6

# Define model parameters
# Create Dataset
listdir = tf.io.gfile.glob(globPatterns)
print(len(listdir))
fileList = np.sort(listdir)
sortedList = np.sort(listdir)
print(sortedList[:5])
np.random.seed(seed)
np.random.shuffle(fileList)


5811
['/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_405.csv'
 '/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_406.csv'
 '/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_407.csv'
 '/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_408.csv'
 '/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_409.csv']


In [ ]:
nFiles = len(fileList)
nPerRecord = 100
start = 0
filePathWrite = '/content/drive/My Drive/Data/Detection/Testing/'
# Read start and beginning from file register
with open(filePathWrite+"FileRegister.csv",'r') as f:
    register = np.genfromtxt(f, delimiter=',').astype(int)
if not np.shape(register) == (3,):
    start = register[-1,1]
    fileIndex = register[-1,2] + 1
else:
    start = 0
    fileIndex = 0

end = start + nPerRecord
tS = time.perf_counter()
durAverage = 0
fileNameWrite = "TestCorrected_"
while end  < nFiles:
    print("Reading from {0} to {1}".format(start,end) )
    fileNameRead = fileList[start]
    data = process(fileNameRead)
    filesLeft = end < nFiles
    for fileNameRead in fileList[start+1: end if filesLeft else None]:
        data = np.concatenate((data,process(fileNameRead)),axis = 0)

    # Write TFRecord file 
    fileName = fileNameWrite + str(fileIndex) + ".tfrecord"
    create_tfrecord(filePathWrite,fileName,data)
    # Uodate File Register
    with open(filePathWrite+'FileRegister.csv','a') as f:
        update = str(start)+ ',' + str(end) + ','+ str(fileIndex) + '\n' 
        f.write(update)
    start = end
    end = start + nPerRecord
    tE = time.perf_counter()
    dur = tE-tS
    if durAverage == 0 :
        durAverage = dur
    else:
        durAverage = 0.9*durAverage +0.1*dur 
    tS = tE
    fileIndex += 1
    print("Written File " + fileName)
    print("Average time per file: {}".format(durAverage))
    print("Estimated minutes left: {}".format((nFiles-start)/(nPerRecord/durAverage)/60))



In [ ]:

# Read and print data:
sess = tf.InteractiveSession()
filePathCheck = filePathWrite
fileNameCheck = 'Test_0.tfrecord'
fileNameCheck2 = 'Test_1.tfrecord'
# Read TFRecord file
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer([filePathCheck+fileNameCheck])
filename_queue2 = tf.train.string_input_producer([filePathCheck+fileNameCheck2])
_, serialized_example = reader.read(filename_queue)
_, serialized_example2 = reader.read(filename_queue2)
# Define features
read_features = {
    'x1': tf.VarLenFeature(dtype=tf.float32),
    'y1': tf.VarLenFeature(dtype=tf.float32),
    'z1': tf.VarLenFeature(dtype=tf.float32),
    'vx1': tf.VarLenFeature(dtype=tf.float32),
    'vy1': tf.VarLenFeature(dtype=tf.float32),
    'vz1':tf.VarLenFeature(dtype=tf.float32),
    'x2': tf.VarLenFeature(dtype=tf.float32),
    'y2': tf.VarLenFeature(dtype=tf.float32),
    'z2': tf.VarLenFeature(dtype=tf.float32),
    'vx2': tf.VarLenFeature(dtype=tf.float32),
    'vy2': tf.VarLenFeature(dtype=tf.float32),
    'vz2':tf.VarLenFeature(dtype=tf.float32),
    'x3': tf.VarLenFeature(dtype=tf.float32),
    'y3': tf.VarLenFeature(dtype=tf.float32),
    'z3': tf.VarLenFeature(dtype=tf.float32),
    'vx3': tf.VarLenFeature(dtype=tf.float32),
    'vy3': tf.VarLenFeature(dtype=tf.float32),
    'vz3':tf.VarLenFeature(dtype=tf.float32),
    'x4': tf.VarLenFeature(dtype=tf.float32),
    'y4': tf.VarLenFeature(dtype=tf.float32),
    'z4': tf.VarLenFeature(dtype=tf.float32),
    'vx4': tf.VarLenFeature(dtype=tf.float32),
    'vy4': tf.VarLenFeature(dtype=tf.float32),
    'vz4':tf.VarLenFeature(dtype=tf.float32),
    'label': tf.VarLenFeature(dtype=tf.int64),
    'time': tf.VarLenFeature(dtype=tf.int64),
    'sats': tf.VarLenFeature(dtype=tf.int64)}

# Extract features from serialized data
read_data = tf.parse_single_example(serialized=serialized_example,
                                    features=read_features)
read_data2 = tf.parse_single_example(serialized=serialized_example2,
                                    features=read_features)
# Many tf.train functions use tf.train.QueueRunner,
# so we need to start it before we read
tf.train.start_queue_runners(sess)

# Print features
x1read = read_data['x1'].eval()
x1read2 = read_data2['x1'].eval()
print(x1read)
print(x1read2)

In [ ]:
filePathWrite = '/content/drive/My Drive/Data/Detection/Testing/'

listFileName = "openFaultIntensityList_Sorted.csv"
listFileNameContent = pd.read_csv(filePathWrite + listFileName,names = ['Names','Intensity'])
fileList = listFileNameContent['Names'].to_numpy()
paramList = listFileNameContent['Intensity'].to_numpy()
fileList = fileList[0::2]
paramList = paramList[0::2]
nFiles = len(fileList)
start = 0

# Read start and beginning from file register
with open(filePathWrite+"FileRegisterOpen.csv",'r') as f:
    register = np.genfromtxt(f, delimiter=',').astype(int)
if not np.shape(register) == (2,):
    start = register[-1,0]+1
else:
    start = 0
print(fileList[1][-10:-8])

durAverage = 0
fileNameWrite = "TestFault_2_"
tS = time.perf_counter()

for i,fTup in enumerate(zip(fileList[start:],paramList[start:])):
    fileNameRead = fTup[0]
    fParam = fTup[1]
    data = process(fileNameRead)
    print(data)
    # Write TFRecord file 
    fileName = fileNameWrite + fileNameRead[-10:-8]+'_' + '{:.8f}'.format(fParam)+ ".tfrecord"
    create_tfrecord(filePathWrite,fileName,data)

    # Update File Register
    with open(filePathWrite+'FileRegisterOpen.csv','a') as f:
        update = str(start + i) +','+ fileName +'\n' 
        f.write(update)

    tE = time.perf_counter()
    dur = tE-tS
    if durAverage == 0 :
        durAverage = dur
    else:
        durAverage = 0.9*durAverage +0.1*dur 
    tS = tE
    fileIndex += 1
    print("Written File " + fileName)
    print("Average time per file: {}".format(durAverage))
    print("Estimated minutes left: {}".format((nFiles-i)/(1/durAverage)/60))


In [ ]:
filePathWrite = '/content/drive/My Drive/Data/Detection/Testing/'

fileList = tf.io.gfile.glob(nfPattern)
fileList = np.sort(fileList)
nFiles = len(fileList)
start = 0

# Read start and beginning from file register
with open(filePathWrite+"FileRegisterNF.csv",'r') as f:
    register = np.genfromtxt(f, delimiter=',').astype(int)
if not np.shape(register) == (2,):
    start = register[-1,0]+1
else:
    start = 0
print(start)

3


In [ ]:


durAverage = 0
fileNameWrite = "TestFault_NoFault_"
tS = time.perf_counter()

for i,fileNameRead in enumerate(fileList[start:]):
    data = process(fileNameRead)
    print(data)
    # Write TFRecord file 
    fileName = fileNameWrite + fileNameRead[len(path)+14:-4]+ ".tfrecord"
    create_tfrecord(filePathWrite,fileName,data)

    # Update File Register
    with open(filePathWrite+'FileRegisterNF.csv','a') as f:
        update = str(start + i) +','+ fileName +'\n' 
        f.write(update)

    tE = time.perf_counter()
    dur = tE-tS
    if durAverage == 0 :
        durAverage = dur
    else:
        durAverage = 0.9*durAverage +0.1*dur 
    tS = tE

    print("Written File " + fileName)
    print("Average time per file: {}".format(durAverage))
    print("Estimated minutes left: {}".format((nFiles-i-start)/(1/durAverage)/60))


Streaming output truncated to the last 5000 lines.
 [-3.9005e-01  2.1815e-01 -2.5698e-01 ...  0.0000e+00  5.9000e+03
   5.0000e+00]
 [-4.0413e-01  2.2096e-01 -1.9756e-01 ...  0.0000e+00  5.9010e+03
   5.0000e+00]]
Written File TestFault_NoFault_24688.tfrecord
Average time per file: 2.0302083957161265
Estimated minutes left: 10.590920464319128
[[ 3.4409e-01  5.8353e-02  4.7029e-01 ...  0.0000e+00  3.0000e+02
   0.0000e+00]
 [ 3.1488e-01  1.7408e-03  4.8610e-01 ...  0.0000e+00  3.0100e+02
   0.0000e+00]
 [ 3.2382e-01  3.5055e-02  4.3387e-01 ...  0.0000e+00  3.0200e+02
   0.0000e+00]
 ...
 [-4.3855e-01  2.0116e-01 -2.5372e-01 ...  0.0000e+00  5.8990e+03
   5.0000e+00]
 [-4.3160e-01  1.8477e-01 -2.3310e-01 ...  0.0000e+00  5.9000e+03
   5.0000e+00]
 [-4.5380e-01  2.4185e-01 -2.1177e-01 ...  0.0000e+00  5.9010e+03
   5.0000e+00]]
Written File TestFault_NoFault_24689.tfrecord
Average time per file: 2.0406822269445053
Estimated minutes left: 10.611547580111427
[[ 2.2293e-01  1.8562e-01  5.274

In [ ]:
fileNameTest = "/content/drive/My Drive/DataRaw/Detection/Testing/DataClosedFault4N_11_430.csv"
data = process(fileNameTest)
plt.plot(data[:,24])